# Notes:


Downloaded Weber County Parcels from [here](https://opendata.gis.utah.gov/datasets/utah-weber-county-parcels-lir?geometry=-112.599%2C41.075%2C-111.315%2C41.436)

- [ ] use overlay to pickup open space (parks dataset, land ownership)
- [ ] overlay hospital points to pick up hospital parcels
- [ ] overlay retail centers point layer
- [ ] dws job points code more commercial parcels as office or retail
- [ ] apply address points to non owned multifamily to get a unit count of apartments/units

In [1]:
import arcpy
import os
import pandas as pd
from arcgis import GIS
import numpy as np
from arcgis.features import GeoAccessor, GeoSeriesAccessor
arcpy.env.overwriteOutput = True

# show all columns
pd.options.display.max_columns = None

C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-c

C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyO

In [2]:
# output folder
outputs = '.\\Outputs'

# store path to gdb
gdb = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb'

In [3]:
# Copy parcel data
parcels = os.path.join(gdb, 'Parcels_Weber_LIR')
parcels_copy = arcpy.FeatureClassToFeatureClass_conversion(parcels, gdb, 'parcels_temp')

In [4]:
# load gdb into pandas dataframe
sdf = pd.DataFrame.spatial.from_featureclass(parcels_copy)

In [5]:
# inspect the shape
sdf.shape

(107670, 28)

In [6]:
# preview last 5 records
sdf.tail(5)

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE
107665,107666,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430009,086430009,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06347616599999, 41.191944920..."
107666,107667,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430003,086430003,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06385715399995, 41.192322325..."
107667,107668,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430004,086430004,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06377360599998, 41.192322016..."
107668,107669,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430002,086430002,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06394070099998, 41.192322633..."
107669,107670,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430001,086430001,,,,0,0.0,0.0,0.0,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06402424999999, 41.192322941..."


In [7]:
# count each value in property class field
sdf["PROP_CLASS"].value_counts()

111- SNGL FAM RES              63569
                               23432
119- PUD                        6762
116- CONDO                      3788
500- COMMERCIAL                 3124
112- DUPLEX                     1530
200-INDUSTRIAL                   903
952- EXEMPT - COM                642
113- 3-4 UNITS                   629
121-OUT BLDGS                    546
117- RECREATIONAL                543
812- PROD FARM                   510
108- SFR on COMM Zone            330
911- RES LOT                     266
114- 5-9 UNITS                   216
951- EXEMPT - RES                164
115- 10+ UNITS                   150
901- VAC - RES - DEV             134
999- UNDEVELOPED VL              103
957-RELATED PARCEL                87
917- REC-VAC-LOT                  55
502-COMM LAND with SFR            48
922- PUD LOT                      44
919- PUD-COMMON                   31
118- MOBILE HOME                  25
508- Both RES & COMM IMPRVD       12
902- C/I VACANT                   11
5

In [8]:
# query for single family parcels
sf_boolean = sdf['PROP_CLASS']=='111- SNGL FAM RES'

# add a column to indicate if parcel is single family
sf_parcels = sdf[sf_boolean]
sdf['isSingleFamily'] = sf_boolean
sdf

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE,isSingleFamily
0,1,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,153040007,153040007,2494 S 4150 W,,,516,567000.0,101974.0,0.97,111- SNGL FAM RES,None,,Hunter Place,2114.0,None,1.0,None,2003,2011,Frame Masonry Veneer,"{""rings"": [[[-112.07847957699994, 41.221276599...",True
1,2,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,140840011,140840011,1538 27TH ST,Ogden,,25,251000.0,55782.0,0.20,111- SNGL FAM RES,None,,Denver Place Add,1038.0,None,1.0,None,1926,1975,Frame Masonry Veneer,"{""rings"": [[[-111.93875957899996, 41.216565272...",True
2,3,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,081570007,081570007,4737 S 3600 W,Roy,,40,287000.0,71205.0,0.20,111- SNGL FAM RES,None,,,1488.0,None,1.0,None,1981,2001,Frame Vinyl,"{""rings"": [[[-112.06733202899994, 41.177368593...",True
3,4,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,090750194,090750194,5234 S 4950 W,Hooper,,31,483000.0,125200.0,1.26,111- SNGL FAM RES,None,,,2474.0,None,2.0,None,1997,2007,Frame Stucco,"{""rings"": [[[-112.098193418, 41.16882466400005...",True
4,5,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,171430003,171430003,524 W 2200 N,Harrisville,,20,226000.0,44507.0,0.14,111- SNGL FAM RES,None,,,792.0,None,1.0,None,1986,2003,Frame Vinyl,"{""rings"": [[[-111.99104493199997, 41.298192436...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107665,107666,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430009,086430009,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06347616599999, 41.191944920...",False
107666,107667,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430003,086430003,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06385715399995, 41.192322325...",False
107667,107668,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430004,086430004,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06377360599998, 41.192322016...",False
107668,107669,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430002,086430002,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06394070099998, 41.192322633...",False


In [9]:
# create empty column for building type attribute
sdf['BUILDING_TYPE'] = ''
sdf

,OBJECTID,COUNTY_NAME,COUNTY_ID,ASSESSOR_SRC,BOUNDARY_SRC,DISCLAIMER,CURRENT_ASOF,PARCEL_ID,SERIAL_NUM,PARCEL_ADD,PARCEL_CITY,TAXEXEMPT_TYPE,TAX_DISTRICT,TOTAL_MKT_VALUE,LAND_MKT_VALUE,PARCEL_ACRES,PROP_CLASS,PRIMARY_RES,HOUSE_CNT,SUBDIV_NAME,BLDG_SQFT,BLDG_SQFT_INFO,FLOORS_CNT,FLOORS_INFO,BUILT_YR,EFFBUILT_YR,CONST_MATERIAL,SHAPE,isSingleFamily,BUILDING_TYPE
0,1,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,153040007,153040007,2494 S 4150 W,,,516,567000.0,101974.0,0.97,111- SNGL FAM RES,None,,Hunter Place,2114.0,None,1.0,None,2003,2011,Frame Masonry Veneer,"{""rings"": [[[-112.07847957699994, 41.221276599...",True,
1,2,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,140840011,140840011,1538 27TH ST,Ogden,,25,251000.0,55782.0,0.20,111- SNGL FAM RES,None,,Denver Place Add,1038.0,None,1.0,None,1926,1975,Frame Masonry Veneer,"{""rings"": [[[-111.93875957899996, 41.216565272...",True,
2,3,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,081570007,081570007,4737 S 3600 W,Roy,,40,287000.0,71205.0,0.20,111- SNGL FAM RES,None,,,1488.0,None,1.0,None,1981,2001,Frame Vinyl,"{""rings"": [[[-112.06733202899994, 41.177368593...",True,
3,4,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,090750194,090750194,5234 S 4950 W,Hooper,,31,483000.0,125200.0,1.26,111- SNGL FAM RES,None,,,2474.0,None,2.0,None,1997,2007,Frame Stucco,"{""rings"": [[[-112.098193418, 41.16882466400005...",True,
4,5,Weber,29,http://www.webercountyutah.gov/assessor/,http://www.webercountyutah.gov/Recorder_Surveyor/,http://www.utah.gov/disclaimer.html,2020-06-01,171430003,171430003,524 W 2200 N,Harrisville,,20,226000.0,44507.0,0.14,111- SNGL FAM RES,None,,,792.0,None,1.0,None,1986,2003,Frame Vinyl,"{""rings"": [[[-111.99104493199997, 41.298192436...",True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107665,107666,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430009,086430009,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06347616599999, 41.191944920...",False,
107666,107667,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430003,086430003,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06385715399995, 41.192322325...",False,
107667,107668,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430004,086430004,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06377360599998, 41.192322016...",False,
107668,107669,Weber,,,,http://www.utah.gov/disclaimer.html,NaT,086430002,086430002,,,,0,0.0,0.0,0.00,,None,,,0.0,None,0.0,None,0,0,,"{""rings"": [[[-112.06394070099998, 41.192322633...",False,


In [10]:
# st building type to 1 where is single family equals True
sdf['BUILDING_TYPE'] = np.where(sdf['isSingleFamily'] == True, 1, "")
sdf[['isSingleFamily','BUILDING_TYPE']].head(10)

,isSingleFamily,BUILDING_TYPE
0,True,1
1,True,1
2,True,1
3,True,1
4,True,1
5,True,1
6,True,1
7,True,1
8,True,1
9,True,1


### Notes
- duplicate process for commercial unit groupings...

In [11]:
# inputs
taz_shp = '.\\Inputs\\TAZ.shp'
parcels = '.\\Inputs\\Utah_Weber_County_Parcels_LIR.gdb\\Parcels_Weber_LIR'

In [12]:
# create output gdb
gdb = os.path.join(outputs, "weber.gdb")
if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs, "weber.gdb")
    
# create second output gdb
gdb2 = os.path.join(outputs, "weber2.gdb")
if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs, "weber2.gdb")

In [13]:
# select parcels within modeling area
parcels_layer = arcpy.MakeFeatureLayer_management(parcels, 'parcels') 
arcpy.SelectLayerByLocation_management(parcels_layer, 'HAVE_THEIR_CENTER_IN', taz_shp)
parcels_for_modeling = arcpy.FeatureClassToFeatureClass_conversion(parcels_layer, gdb, '_01_parcels_for_modeling')

# add rings and part count fields
arcpy.AddField_management(parcels_for_modeling, 'ORIG_PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'ORIG_RINGS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'NOW_PARTS', 'LONG')
arcpy.AddField_management(parcels_for_modeling, 'NOW_RINGS', 'LONG')

# Use cursor to calculate parts and rings
fields = ["shape@", 'ORIG_PARTS', 'ORIG_RINGS', 'NOW_PARTS', 'NOW_RINGS']
with arcpy.da.UpdateCursor(parcels_for_modeling, fields) as cursor:
    for row in cursor:
        shape = row[0]
        parts = shape.partCount
        rings = shape.boundary().partCount
        
        row[1] = parts
        row[2] = rings
        row[3] = parts
        row[4] = rings
        
        cursor.updateRow(row)
        
# add a tag field for parcel type
arcpy.AddField_management(parcels_for_modeling, 'TAG', 'TEXT')

# create the master layer for selection
parcels_for_modeling_layer = arcpy.MakeFeatureLayer_management(parcels_for_modeling, 'parcels_for_modeling_layer')

# get a count of all parcels
count_all = arcpy.GetCount_management(parcels_for_modeling_layer)
print("The total amount of parcels is {}".format(count_all))

# create centroids for all parcels
parcels_for_modeling_centroids = arcpy.FeatureToPoint_management(parcels_for_modeling_layer, os.path.join(gdb, '_01b_parcels_for_modeling_centroids'), "INSIDE")

The total amount of parcels is 96978


# Single Family homes

- [ ] Need to handle miscoded puds

In [14]:
tag="single_family"
query = (''' PROP_CLASS = '111- SNGL FAM RES' Or PROP_CLASS = '911- RES LOT' Or PROP_CLASS = '901- VAC - RES - DEV' or 
                PROP_CLASS = '917- REC-VAC-LOT' ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)

expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

# convert to feature class
single_family =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_02_single_family')

There were "61358" single_family parcels selected out of 96978


# Multi Family homes

In [15]:
tag="multi_family"
query = (''' PROP_CLASS LIKE '%112-%' or PROP_CLASS LIKE '%113-%' or PROP_CLASS LIKE '%114-%' or PROP_CLASS LIKE '%115-%' 
                or PROP_CLASS LIKE '%118-%' or PROP_CLASS = '300- MULTIPLE HOUSING'  ''')
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

multi_family =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_03_multi_family')


There were "2555" multi_family parcels selected out of 96978


# Industrial

In [16]:
tag="industrial"
query=(""" PROP_CLASS LIKE '200-INDUSTRIAL%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

industrial =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_04_industrial')

There were "900" industrial parcels selected out of 96978


# Commercial

In [17]:
tag="commercial"
query=""" PROP_CLASS = '500- COMMERCIAL' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

commercial = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_05_commercial')

There were "3024" commercial parcels selected out of 96978


# Retail

In [18]:
tag="retail"
query = (""" PROP_CLASS = '500- COMMERCIAL' And (SUBDIV_NAME LIKE '%MALL%' Or SUBDIV_NAME LIKE '%CARS%' Or SUBDIV_NAME 
                LIKE '%RETAIL%' OR SUBDIV_NAME LIKE '%SALE%' OR SUBDIV_NAME LIKE '%MARKET%') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)
# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

retail = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_06_retail')

There were "40" retail parcels selected out of 96978


# Office

In [19]:
tag="office"
query= (""" PROP_CLASS = '500- COMMERCIAL' And (SUBDIV_NAME LIKE '%BUS%' Or SUBDIV_NAME LIKE '%OFFICE%' Or 
                SUBDIV_NAME LIKE '%BANK%') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

office = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_07_office')

There were "51" office parcels selected out of 96978


# Government

In [20]:
tag="government"
query = (""" TAXEXEMPT_TYPE like '501%' or TAXEXEMPT_TYPE like '502%' or TAXEXEMPT_TYPE like '505%' or TAXEXEMPT_TYPE like 
                '506%' or TAXEXEMPT_TYPE like '507%' or TAXEXEMPT_TYPE like '508%' or TAXEXEMPT_TYPE like '509%' or 
                TAXEXEMPT_TYPE like '510%' or TAXEXEMPT_TYPE like '511%' or TAXEXEMPT_TYPE like '512%' or TAXEXEMPT_TYPE like 
                '513%' or TAXEXEMPT_TYPE like '514%' or TAXEXEMPT_TYPE like '515%' or TAXEXEMPT_TYPE like '516%' or 
                TAXEXEMPT_TYPE like '517%' or TAXEXEMPT_TYPE like '400%' or TAXEXEMPT_TYPE like '450%' or TAXEXEMPT_TYPE like 
                '200%' or TAXEXEMPT_TYPE like '100%' or TAXEXEMPT_TYPE like '110%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

government = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_08_government')

There were "3597" government parcels selected out of 96978


# Agriculture

In [21]:
tag="agriculture"
query = """ PROP_CLASS = '811- FARM ANIMALS' Or PROP_CLASS = '812- PROD FARM' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

agriculture = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_09_agriculture')

There were "404" agriculture parcels selected out of 96978


# Education

- [ ] Check for private schools in Weber County e.g. St. Josephs

In [22]:
tag="education"
query = (""" TAXEXEMPT_TYPE = '310-Ogden City Schools' Or TAXEXEMPT_TYPE = '320-Weber County Schools' Or 
                TAXEXEMPT_TYPE = '340-WSU' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

education =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_10_education')

There were "301" education parcels selected out of 96978


# Churches

In [23]:
tag="churches"
query = (""" TAXEXEMPT_TYPE LIKE '801-%' OR TAXEXEMPT_TYPE LIKE '803-%' OR TAXEXEMPT_TYPE LIKE '804-%' OR 
                TAXEXEMPT_TYPE LIKE '805-%' OR TAXEXEMPT_TYPE LIKE '806-%' """) 
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

churches =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_11_churches')

There were "488" churches parcels selected out of 96978


# Healthcare

In [24]:
tag = 'healthcare'
query = (""" PROP_CLASS = '500- COMMERCIAL' And (SUBDIV_NAME LIKE '%HOSP%' Or SUBDIV_NAME LIKE '%CLINIC%' Or 
                SUBDIV_NAME LIKE '%MEDICAL%') """)

arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

healthcare = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_13_healthcare')

There were "6" healthcare parcels selected out of 96978


# Corridors

In [25]:
tag="corridors"
query = (""" TAXEXEMPT_TYPE like '210%' or TAXEXEMPT_TYPE like '220%' or TAXEXEMPT_TYPE like '402%' or 
                TAXEXEMPT_TYPE like '900%' or TAXEXEMPT_TYPE like '611%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

corridors = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_14_corridors')

There were "1536" corridors parcels selected out of 96978


# Utilities

In [26]:
tag="utilities"
query= (""" TAXEXEMPT_TYPE like '601%' or TAXEXEMPT_TYPE like '602%' or TAXEXEMPT_TYPE like '603%' or 
                TAXEXEMPT_TYPE like '604%' or TAXEXEMPT_TYPE like '605%' or TAXEXEMPT_TYPE like '607%' or 
                TAXEXEMPT_TYPE like '609%' or TAXEXEMPT_TYPE like '995%' """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
expression = "createTag(!TAG!, '{}')".format(tag) 
codeblock = """
def createTag(tag_current, tag_new):
    if tag_current == None or tag_current == '':
        return tag_new
    else:
        return tag_current + ',' + tag_new"""

arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
                                code_block=codeblock)

# count the selected features
count = arcpy.GetCount_management(parcels_for_modeling_layer)
print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

utilities = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_15_utilities')

There were "507" utilities parcels selected out of 96978


# Mixed Use

In [27]:
# tag = "mixed_use"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

# mixed_use = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_16_mixed_use')

# Group Quarters

In [28]:
# tag = "group_quarters"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

# group_quarters = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_16_group_quarters')

# Common Areas

In [29]:
# tag = "common_areas"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

# common_areas = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_17_common_areas')

# Other Building Types

In [30]:
# tag = "other"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

# other = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_16_other')

# Open space

In [31]:
# tag = "open_space"

# query= ("""  """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# expression = "createTag(!TAG!, '{}')".format(tag) 
# codeblock = """
# def createTag(tag_current, tag_new):
#     if tag_current == None or tag_current == '':
#         return tag_new
#     else:
#         return tag_current + ',' + tag_new"""

# arcpy.CalculateField_management(parcels_for_modeling_layer, field='TAG', expression=expression,expression_type="PYTHON3",
#                                 code_block=codeblock)

# # count the selected features
# count = arcpy.GetCount_management(parcels_for_modeling_layer)
# print('There were "{}" {} parcels selected out of {}'.format(count, tag, count_all))

# open_space = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_17_open_space')

In [32]:
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'CLEAR_SELECTION')
parcels_tagged = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb2, '_01_parcels_tagged')

# Owned Unit Groupings

### Step 1: Create the polygons

In [33]:
# filter for common areas, some small office parks and a few other strange cases may be present
query = """ TAXEXEMPT_TYPE = '700-Common Area' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
common_areas =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_02_common_areas')

# fill in holes via eliminate polygon part
common_areas_filled = arcpy.EliminatePolygonPart_management(common_areas, os.path.join(gdb, '_03_common_areas_filled'),
                                                      condition='PERCENT', part_area_percent=60, part_option='CONTAINED_ONLY')

In [34]:
# get common areas, condos, and puds
query = (""" (PROP_CLASS IN ('116- CONDO', '119- PUD' )) OR
(TAXEXEMPT_TYPE = '700-Common Area') """)
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
common_areas_condos_puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, 
                                                                        '_04_common_areas_condos_puds')

common_areas_condos_puds_lyr = arcpy.MakeFeatureLayer_management(common_areas_condos_puds, 'common_areas_condos_puds_lyr')

# get condos and puds
query = """ PROP_CLASS IN ('116- CONDO', '119- PUD' ) AND TAXEXEMPT_TYPE <> '700-Common Area' """
arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
condos_and_puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_05a_condos_and_puds')

# create condos and pud (CAP) centroids
cap_centroids = arcpy.FeatureToPoint_management(condos_and_puds, os.path.join(gdb, '_05b_condos_and_puds_centroids'), "INSIDE")

In [35]:
# load centroids into dataframe
centroids_sdf = pd.DataFrame.spatial.from_featureclass(cap_centroids)

# get unique sub division values
sd_list = list(set(centroids_sdf['SUBDIV_NAME'].to_list()))
sd_list.sort()

# use dictionary comprehension
sd_dict = {sd: num for num, sd in enumerate(sd_list)}
#sd_dict

# map new values to new field
centroids_sdf['SUBDIV_NUM'] = centroids_sdf['SUBDIV_NAME'].map(sd_dict)
centroids_sdf[['SUBDIV_NAME','SUBDIV_NUM']].head(10)

#export back to gdb
cap_centroids = centroids_sdf.spatial.to_featureclass(location=os.path.join(gdb,'_05b_condos_and_puds_centroids'))

In [36]:
# get parcels from filled common areas that overlap with centroids
common_areas_filled_lyr = arcpy.MakeFeatureLayer_management(common_areas_filled, 'common_areas_filled_lyr') 

arcpy.SelectLayerByLocation_management(in_layer=common_areas_filled_lyr , overlap_type="CONTAINS", 
                                       select_features=cap_centroids, selection_type='NEW_SELECTION')

common_areas_containing_units = arcpy.FeatureClassToFeatureClass_conversion(common_areas_filled_lyr, gdb, 
                                                                            '_06_common_areas_containing_units')

# get parcels (from parcels for modeling) that do not overlap with filled parcels that contained units
arcpy.SelectLayerByLocation_management(in_layer=common_areas_condos_puds_lyr,overlap_type="WITHIN",
                                       select_features=common_areas_containing_units,
                                       selection_type='NEW_SELECTION', invert_spatial_relationship='INVERT')

leftover_parcels = arcpy.FeatureClassToFeatureClass_conversion(common_areas_condos_puds_lyr, gdb, 
                                                                            '_07_leftover_parcels')

# dissolve these parcels unchecking create multipart
leftover_parcels_dissolved = arcpy.Dissolve_management(leftover_parcels, os.path.join(gdb,"_08_leftover_parcels_dissolved"),
                                                       multi_part="SINGLE_PART")

# fill in holes via eliminate polygon part
leftover_parcels_filled = arcpy.EliminatePolygonPart_management(leftover_parcels_dissolved,
                                                                os.path.join(gdb, '_09_leftover_parcels_filled'),
                                                                condition='PERCENT', part_area_percent=60, 
                                                                part_option='CONTAINED_ONLY')

In [37]:
# merge first set of parcels with dissolved parcels
owned_unit_groupings_bdry = arcpy.Merge_management([common_areas_containing_units, leftover_parcels_filled], 
                                              os.path.join(gdb, "_10_owned_unit_groupings_bdry"))
# delete all fields except required
fieldObjList = arcpy.ListFields(owned_unit_groupings_bdry)
fieldNameList = [field.name  for field in fieldObjList if not field.required]      
arcpy.DeleteField_management(owned_unit_groupings_bdry, fieldNameList)        

<Result '.\\Outputs\\weber.gdb\\_10_owned_unit_groupings_bdry'>

In [38]:
# run spatial join on common areas - summarize market value & acreage
target_features = owned_unit_groupings_bdry
join_features = cap_centroids
output_features = os.path.join(gdb, "_11_oug_sj")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# total market value
fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Sum'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# parcel acres
fieldindex = fieldmappings.findFieldMapIndex('SUBDIV_NUM')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'Mode'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# Set field aggregation rule using loop
# TOTAL_MKT_VALUE = Total Market Value
# PARCEL_ACRES = Parcel Acres
# fields_list = ['TOTAL_MKT_VALUE', 'PARCEL_ACRES']
# for field in fields_list:
#     modFieldMapping(fieldmappings, field, 'Sum')

oug_sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "CONTAINS")

# dissolve these parcels unchecking create multipart
oug_sj_dissolve = arcpy.Dissolve_management(oug_sj,os.path.join(gdb,"_12_oug_sj_dissolve"),dissolve_field=["SUBDIV_NUM"],
                                            multi_part="SINGLE_PART")


In [39]:
# identify common areas that don't have a parcel centroid
oug_lyr = arcpy.MakeFeatureLayer_management(oug_sj_dissolve, 'oug_lyr')

arcpy.SelectLayerByLocation_management(in_layer=oug_lyr , overlap_type="CONTAINS", 
                                       select_features=cap_centroids, selection_type='NEW_SELECTION', 
                                       invert_spatial_relationship='INVERT')

empty_ougs = arcpy.FeatureClassToFeatureClass_conversion(oug_lyr, gdb, '_13_empty_ougs')



# identify and count parcels that were grouped by dissolving final polys into single polygon
oug_mp = arcpy.Dissolve_management(oug_sj,os.path.join(gdb,"_14_oug_multipart"))

arcpy.SelectLayerByLocation_management(in_layer=parcels_for_modeling_layer , overlap_type="WITHIN", 
                                       select_features=oug_mp, selection_type='NEW_SELECTION')

parcels_within_ougs = arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, '_15_parcels_within_ougs')
count = arcpy.GetCount_management(parcels_within_ougs)
print('There were "{}" parcels within OUGs selected out of {}'.format(count, count_all))



There were "11781" parcels within OUGs selected out of 96978


### PAUSE - Review OUGs

In [40]:
# # export centroids for pass 2
# centroids_pass2 = arcpy.FeatureClassToFeatureClass_conversion(cap_centroids_lyr, gdb, 'centroids_pass2')

# # export centroids for pass 1 by inverting
# arcpy.SelectLayerByAttribute_management(cap_centroids_lyr, "SWITCH_SELECTION")
# centroids_pass1 = arcpy.FeatureClassToFeatureClass_conversion(cap_centroids_lyr, gdb, 'centroids_pass1')

In [41]:
# # get common areas, condos, and puds
# query = (""" (PROP_CLASS IN ('116- CONDO', '119- PUD' ) AND TAXEXEMPT_TYPE <> '700-Common Area') OR
# (TAXEXEMPT_TYPE = '700-Common Area') """)
# arcpy.SelectLayerByAttribute_management(parcels_for_modeling_layer, 'NEW_SELECTION', query)
# common_areas_condos_puds =  arcpy.FeatureClassToFeatureClass_conversion(parcels_for_modeling_layer, gdb, 'common_areas_condos_puds')

# # Merge smaller parcels to main common area
# common_areas_condos_puds_layer = arcpy.MakeFeatureLayer_management(common_areas_condos_puds, 'parcels_for_modeling_layer')
# query = (""" PROP_CLASS IN ('116- CONDO', '119- PUD' ) AND TAXEXEMPT_TYPE <> '700-Common Area' """)
# arcpy.SelectLayerByAttribute_management(common_areas_condos_puds_layer, 'NEW_SELECTION', query)
# common_areas_full = arcpy.Eliminate_management(common_areas_condos_puds_layer, os.path.join(gdb, 'common_areas_full'), 
#                                                selection='LENGTH')

In [42]:
# # sets the field map aggregation rule for a provided field/fieldmapping
# # example merge rules: sum, mean, first, min, join, count, etc...
# def modFieldMapping(fieldMappingsObject, field, mergeRule):
#     fieldindex = fieldMappingsObject.findFieldMapIndex(field)
#     fieldmap = fieldmappings.getFieldMap(fieldindex)
#     fieldmap.mergeRule = mergeRule
#     fieldMappingsObject.replaceFieldMap(fieldindex, fieldmap)

### Step 2: aggregate the attributes

In [43]:
# # run spatial join on common areas - summarize market value & acreage
# target_features = common_areas_elim
# join_features = cap_centroids
# output_features = os.path.join(gdb, "common_areas_sj_pass1")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # total market value
# fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # Set field aggregation rule using loop
# # TOTAL_MKT_VALUE = Total Market Value
# # PARCEL_ACRES = Parcel Acres
# # fields_list = ['TOTAL_MKT_VALUE', 'PARCEL_ACRES']
# # for field in fields_list:
# #     modFieldMapping(fieldmappings, field, 'Sum')

# arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
#                            fieldmappings, "CONTAINS")




## Spatial join pass #2

In [44]:
# # get list of condos and puds that did not make the first aggregation
# cap_centroids_lyr = arcpy.MakeFeatureLayer_management(cap_centroids, 'cap_centroids_lyr') 
# arcpy.SelectLayerByLocation_management(in_layer=cap_centroids_lyr, overlap_type="COMPLETELY_WITHIN", select_features=common_areas_elim, 
#                                         selection_type='NEW_SELECTION',invert_spatial_relationship='INVERT')

# # export centroids for pass 2
# centroids_pass2 = arcpy.FeatureClassToFeatureClass_conversion(cap_centroids_lyr, gdb, 'centroids_pass2')

# # export centroids for pass 1 by inverting
# arcpy.SelectLayerByAttribute_management(cap_centroids_lyr, "SWITCH_SELECTION")
# centroids_pass1 = arcpy.FeatureClassToFeatureClass_conversion(cap_centroids_lyr, gdb, 'centroids_pass1')

# # Switch them back
# arcpy.SelectLayerByAttribute_management(cap_centroids_lyr, "SWITCH_SELECTION")

# caf_lyr = arcpy.MakeFeatureLayer_management(common_areas_full, 'caf_lyr') 
# arcpy.SelectLayerByLocation_management(in_layer=caf_lyr, overlap_type="CONTAINS", select_features=cap_centroids_lyr, 
#                                         selection_type='NEW_SELECTION')

# # run spatial join on common areas "full" - summarize market value & acreage
# target_features = caf_lyr
# join_features = cap_centroids_lyr
# output_features = os.path.join(gdb, "common_areas_sj_pass2")

# fieldmappings = arcpy.FieldMappings()
# fieldmappings.addTable(target_features)
# fieldmappings.addTable(join_features)

# # total market value
# fieldindex = fieldmappings.findFieldMapIndex('TOTAL_MKT_VALUE')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # parcel acres
# fieldindex = fieldmappings.findFieldMapIndex('PARCEL_ACRES')
# fieldmap = fieldmappings.getFieldMap(fieldindex)
# fieldmap.mergeRule = 'Sum'
# fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# # Set field aggregation rule using loop
# # TOTAL_MKT_VALUE = Total Market Value
# # PARCEL_ACRES = Parcel Acres
# # fields_list = ['TOTAL_MKT_VALUE', 'PARCEL_ACRES']
# # for field in fields_list:
# #     modFieldMapping(fieldmappings, field, 'Sum')

# arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
#                            fieldmappings, "CONTAINS")
